## Importing important libraryes 

In [ ]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
import re
import nltk
#from contractions import contractions_dict
from nltk.corpus import stopwords
from spacy.lang.en.stop_words import STOP_WORDS
from itertools import filterfalse
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report

In [ ]:
data = pd.read_csv("email.csv") 

### Removing unwanted labels and remane the lables

In [ ]:
data.rename(columns={'label_num':'spam'}, inplace=True)
data.drop('label',axis='columns', inplace=True)

In [ ]:
data.drop('Unnamed: 0',axis='columns', inplace=True)

## Remove unwanted words in the data

In [ ]:
def strip_titles(text):
    if "Subject: re :" in text:
        return text[13:]
    elif "Subject: news :" in text:
        return text[15:]
    else:
        return text[8:]

In [ ]:
data['text'] = data['text'].apply(lambda x: strip_titles(x))

### We r going to tolonize all emails

In [ ]:
data['text'] = data['text'].apply(lambda x: word_tokenize(x))

## Normalize = Now we have to convert all tokens into lower case

In [ ]:
def normalize_tokens(list_of_tokens):
    return map(lambda x: x.lower(),list_of_tokens)

In [ ]:
data['text'] = data['text'].apply(lambda x: normalize_tokens(x))


In [ ]:
data['text'] = data['text'].apply(lambda x: list(x))


### Now we r going to remove contractions

In [ ]:
contractions_dict = {"'cause": 'because',"'tis": 'it is',"'twas": 'it was',"I'd": 'I would',"I'd've": 'I would have',"I'll": 'I will',"I'll've": 'I will have',"I'm": 'I am',"I'm'a": 'I am about to',"I'm'o": 'I am going to',"I've": 'I have','I’d': 'I would','I’d’ve': 'I would have','I’ll': 'I will','I’ll’ve': 'I will have','I’m': 'I am','I’m’a': 'I am about to','I’m’o': 'I am going to','I’ve': 'I have','Whatcha': 'What are you',"ain't": 'are not','ain’t': 'are not',"amn't": 'am not','amn’t': 'am not','apr.': 'april',"aren't": 'are not','aren’t': 'are not','aug.': 'august',"can't": 'cannot',"can't've": 'cannot have','can’t': 'cannot','can’t’ve': 'cannot have',"could've": 'could have',"couldn't": 'could not',"couldn't've": 'could not have','couldn’t': 'could not','couldn’t’ve': 'could not have','could’ve': 'could have',"daren't": 'dare not','daren’t': 'dare not',"daresn't": 'dare not','daresn’t': 'dare not',"dasn't": 'dare not','dasn’t': 'dare not','dec.': 'december',"didn't": 'did not','didn’t': 'did not',"doesn't": 'does not','doesn’t': 'does not',"don't": 'do not','don’t': 'do not',"e'er": 'ever','em': 'them',"everyone's": 'everyone is','everyone’s': 'everyone is','e’er': 'ever','feb.': 'february','finna': 'fixing to','gimme': 'give me',"gon't": 'go not','gonna': 'going to','gon’t': 'go not','gotta': 'got to',"hadn't": 'had not',"hadn't've": 'had not have','hadn’t': 'had not','hadn’t’ve': 'had not have',"hasn't": 'has not','hasn’t': 'has not',"haven't": 'have not','haven’t': 'have not',"he'd": 'he would',"he'd've": 'he would have',"he'll": 'he will',"he'll've": 'he will have',"he's": 'he is',"he've": 'he have',"here's": 'here is','here’s': 'here is','he’d': 'he would','he’d’ve': 'he would have','he’ll': 'he will','he’ll’ve': 'he will have','he’s': 'he is','he’ve': 'he have',"how'd": 'how did',"how'd'y": 'how do you',"how'll": 'how will',"how're": 'how are',"how's": 'how is','how’d': 'how did','how’d’y': 'how do you','how’ll': 'how will','how’re': 'how are','how’s': 'how is',"isn't": 'is not','isn’t': 'is not',"it'd": 'it would',"it'd've": 'it would have',"it'll": 'it will',"it'll've": 'it will have',"it's": 'it is','it’d': 'it would','it’d’ve': 'it would have','it’ll': 'it will','it’ll’ve': 'it will have','it’s': 'it is','jan.': 'january','jul.': 'july','jun.': 'june','kinda': 'kind of',"let's": 'let us','let’s': 'let us','luv': 'love',"ma'am": 'madam','mar.': 'march',"may've": 'may have',"mayn't": 'may not','mayn’t': 'may not','may’ve': 'may have','ma’am': 'madam',"might've": 'might have',"mightn't": 'might not',"mightn't've": 'might not have','mightn’t': 'might not','mightn’t’ve': 'might not have','might’ve': 'might have',"must've": 'must have',"mustn't": 'must not',"mustn't've": 'must not have','mustn’t': 'must not','mustn’t’ve': 'must not have','must’ve': 'must have',"ne'er": 'never',"needn't": 'need not',"needn't've": 'need not have','needn’t': 'need not','needn’t’ve': 'need not have','ne’er': 'never','nov.': 'november',"o'": 'of',"o'clock": 'of the clock',"o'er": 'over','oct.': 'october',"ol'": 'old','ol’': 'old',"oughtn't": 'ought not',"oughtn't've": 'ought not have','oughtn’t': 'ought not','oughtn’t’ve': 'ought not have','o’': 'of','o’clock': 'of the clock','o’er': 'over','sep.': 'september',"sha'n't": 'shall not',"shalln't": 'shall not','shalln’t': 'shall not',"shan't": 'shall not',"shan't've": 'shall not have','shan’t': 'shall not','shan’t’ve': 'shall not have','sha’n’t': 'shall not',"she'd": 'she would',"she'd've": 'she would have',"she'll": 'she will',"she's": 'she is','she’d': 'she would','she’d’ve': 'she would have','she’ll': 'she will','she’s': 'she is',"should've": 'should have',"shouldn't": 'should not',"shouldn't've": 'should not have','shouldn’t': 'should not','shouldn’t’ve': 'should not have','should’ve': 'should have',"so's": 'so is',"so've": 'so have',"somebody's": 'somebody is','somebody’s': 'somebody is',"someone's": 'someone is','someone’s': 'someone is',"something's": 'something is','something’s': 'something is','so’s': 'so is','so’ve': 'so have','sux': 'sucks',"that'd": 'that would',"that'd've": 'that would have',"that'll": 'that will',"that're": 'that are',"that's": 'that is','that’d': 'that would','that’d’ve': 'that would have','that’ll': 'that will','that’re': 'that are','that’s': 'that is',"there'd": 'there would',"there'd've": 'there would have',"there'll": 'there will',"there're": 'there are',"there's": 'there is','there’d': 'there would','there’d’ve': 'there would have','there’ll': 'there will','there’re': 'there are','there’s': 'there is',"these're": 'these are','these’re': 'these are',"they'd": 'they would',"they'd've": 'they would have',"they'll": 'they will',"they'll've": 'they will have', "they're": 'they are', "they've": 'they have', 'they’d': 'they would','they’d’ve': 'they would have', 'they’ll': 'they will', 'they’ll’ve': 'they will have', 'they’re': 'they are','they’ve': 'they have', "this's": 'this is', 'this’s': 'this is', "those're": 'those are','those’re': 'those are', "to've": 'to have', 'to’ve': 'to have', 'wanna': 'want to',"wasn't": 'was not', 'wasn’t': 'was not', "we'd": 'we would', "we'd've": 'we would have',"we'll": 'we will', "we'll've": 'we will have', "we're": 'we are', "we've": 'we have',"weren't": 'were not', 'weren’t': 'were not', 'we’d': 'we would', 'we’d’ve': 'we would have','we’ll': 'we will', 'we’ll’ve': 'we will have', 'we’re': 'we are', 'we’ve': 'we have',"what'd": 'what did', "what'll": 'what will', "what'll've": 'what will have',"what're": 'what are', "what's": 'what is', "what've": 'what have', 'what’d': 'what did','what’ll': 'what will', 'what’ll’ve': 'what will have', 'what’re': 'what are', 'what’s': 'what is','what’ve': 'what have', "when's": 'when is', "when've": 'when have', 'when’s': 'when is', 'when’ve': 'when have', "where'd": 'where did',"where're": 'where are', "where's": 'where is', "where've": 'where have', 'where’d': 'where did','where’re': 'where are', 'where’s': 'where is', 'where’ve': 'where have', "which's": 'which is','which’s': 'which is', "who'd": 'who would', "who'd've": 'who would have', "who'll": 'who will',"who'll've": 'who will have', "who're": 'who are', "who's": 'who is', "who've": 'who have','who’d': 'who would', 'who’d’ve': 'who would have', 'who’ll': 'who will', 'who’ll’ve': 'who will have','who’re': 'who are', 'who’s': 'who is', 'who’ve': 'who have', "why'd": 'why did',"why're": 'why are', "why's": 'why is', "why've": 'why have', 'why’d': 'why did','why’re': 'why are', 'why’s': 'why is', 'why’ve': 'why have', "will've": 'will have','will’ve': 'will have', "won't": 'will not', "won't've": 'will not have', 'won’t': 'will not','won’t’ve': 'will not have', "would've": 'would have', "wouldn't": 'would not', "wouldn't've": 'would not have','wouldn’t': 'would not', 'wouldn’t’ve': 'would not have', 'would’ve': 'would have', "y'all": 'you all', "y'all'd": 'you all would', "y'all'd've": 'you all would have',"y'all're": 'you all are', "y'all've": 'you all have', "you'd": 'you would', "you'd've": 'you would have', "you'll": 'you will', "you'll've": 'you shall have', "you're": 'you are', "you've": 'you have','you’d': 'you would', 'you’d’ve': 'you would have', 'you’ll': 'you will', 'you’ll’ve': 'you shall have', 'you’re': 'you are', 'you’ve': 'you have', 'y’all': 'you all', 'y’all’d': 'you all would', 'y’all’d’ve': 'you all would have','y’all’re': 'you all are', 'y’all’ve': 'you all have', '’cause': 'because', '’tis': 'it is', '’twas': 'it was'}

In [ ]:
def contracted_word_expansion(token):
    if token in contractions_dict.keys():
        return contractions_dict[token]
    else:
        return token

In [ ]:
contracted_word_expansion('You don\'t need a library')

In [ ]:
def contractions_expansion(list_of_tokens):
    return map(contracted_word_expansion,list_of_tokens)

In [ ]:
data['text'] = data['text'].apply(lambda x: contractions_expansion(x))

In [ ]:
data['text'] = data['text'].apply(lambda x: list(x))

### Now data came is a non contracted data

In [ ]:
data

### Now we remove unnecessary words

In [ ]:
regex = r'^@[a-zA-z0-9]|^#[a-zA-Z0-9]|\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*|\W+|\d+|<("[^"]*"|\'[^\']*\'|[^\'">])*>|_+|[^\u0000-\u007f]+'

In [ ]:
def waste_word_or_not(token):
    return re.search(regex,token)

In [ ]:
def filter_waste_words(list_of_tokens):
    return filterfalse(waste_word_or_not,list_of_tokens)

In [ ]:
data['text'] = data['text'].apply(lambda x: filter_waste_words(x))

In [ ]:
data['text'] = data['text'].apply(lambda x: list(x))

In [ ]:
data['text']

### Splitting

In [ ]:
def split(list_of_tokens):
    return map(lambda x: re.split(regex,x)[0],list_of_tokens)

In [ ]:
data['text'] = data['text'].apply(lambda x: split(x))

In [ ]:
data['text'] = data['text'].apply(lambda x: list(x))

In [ ]:
data['text']

### We r going to remove stop words

In [ ]:
en_stop_words = list(set(stopwords.words('english')).union(set(STOP_WORDS)))

In [ ]:
len(en_stop_words)

In [ ]:
def is_stopword(token):
    return not(token in en_stop_words or re.search(r'\b\w\b|[^\u0000-\u007f]+|_+|\W+',token))

In [ ]:
def stopwords_removals(list_of_tokens):
    return filter(is_stopword,list_of_tokens)

In [ ]:
data['text'] = data['text'].apply(lambda x: stopwords_removals(x))

In [ ]:
data['text'] = data['text'].apply(lambda x: list(x))

In [ ]:
data['text']

## Now we r going to perform limitization

In [ ]:
def get_wnet_pos_tag(treebank_tag):
    if treebank_tag[1].startswith('J'):
        return (treebank_tag[0],wordnet.ADJ)
    elif treebank_tag[1].startswith('V'):
        return (treebank_tag[0],wordnet.VERB)
    elif treebank_tag[1].startswith('N'):
        return (treebank_tag[0],wordnet.NOUN)
    elif treebank_tag[1].startswith('R'):
        return (treebank_tag[0],wordnet.ADV)
    else:
        (treebank_tag[0],wordnet.NOUN)

In [ ]:
def get_pos_tag(list_of_tokens):
    return map(get_wnet_pos_tag,pos_tag(list_of_tokens))

In [ ]:
data['text'] = data['text'].apply(lambda x: get_pos_tag(x))

In [ ]:
nltk.download('wordnet')

In [ ]:
nltk.download('averaged_perceptron_tagger')

In [ ]:
data['text'] = data['text'].apply(lambda x: list(x))

In [ ]:
data['text']

### Now we r going to apply Limitization

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
def token_lemmatization(token):
    if token == None:
        return ""
    else:
        return lemmatizer.lemmatize(word=token[0],pos=token[1])

In [ ]:
def lemmatization(list_of_tokens):
    if len(list_of_tokens) > 0:
        return map(lambda x: token_lemmatization(x),list_of_tokens)
    else:
        return ""

In [ ]:
data['text'] = data['text'].apply(lambda x: lemmatization(x))

In [ ]:
data['text']

In [ ]:
data['text'] = data['text'].apply(lambda x: list(x))

In [ ]:
vocab = set()
for list_of_tokens in data['text']:
    vocab = vocab.union(set(list_of_tokens))

In [ ]:
vocab = list(vocab)

In [ ]:
vocab.pop(0)

In [ ]:
vocab_dict = dict(zip(vocab,list(range(0,len(vocab)))))

### Now we r joinng tokens

In [ ]:
def join_tokens(list_of_tokens):
    return " ".join(list_of_tokens)

In [ ]:
data['text'] = data['text'].apply(lambda x: join_tokens(x))

In [ ]:
corpus = list()
for email_text in data['text']:
    corpus.append(email_text)

In [ ]:
vectorizer = TfidfVectorizer(vocabulary=vocab_dict)
tf_idf_matrix = vectorizer.fit_transform(corpus)

In [ ]:
tf_idf_matrix = tf_idf_matrix.toarray()

In [ ]:
df = pd.DataFrame(tf_idf_matrix)

In [ ]:
df['spam'] = data['spam']

In [ ]:
pca = PCA(n_components=5000)

In [ ]:
tf_idf_matrix_reduced = pca.fit_transform(tf_idf_matrix)

In [ ]:
df = pd.DataFrame(data=tf_idf_matrix_reduced)

In [ ]:
df['spam'] = data['spam']

In [ ]:
gnb = GaussianNB()

In [ ]:
X_train = df.iloc[:,0:5000]
y_train = df['spam']

In [ ]:
gnb.fit(X=X_train,y=y_train)

In [ ]:
predicted_categories = gnb.predict(X_train)

In [ ]:
print(classification_report(y_true=y_train,y_pred=predicted_categories))